# Acquire the Data

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

For this project, we will be using data in a timeline format. Ultimately, our model will be using a snapshot of the match at the 20 minute mark.

In [2]:
#Grab timeline data
response = requests.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4083906028/timeline/?api_key={api_key}')

In [3]:
#Turn it into json format
data = response.json()

In [4]:
data.keys()

dict_keys(['metadata', 'info'])

In [5]:
data['metadata']

{'dataVersion': '2',
 'matchId': 'NA1_4083906028',
 'participants': ['l5armf8S4PORoTNqEb9UcVkEvDzSYClySmYY4yQFBUU2ciCjRfQJLP9x-6uIMjmeUSm7x8-JnEWQxg',
  'idOuCKMdqN2RMvthmMVRFT8FXZSVnkmji_qWCXptdRpdltnNyXkEgNEj5Kl52iiJAVmGh6TFsZafdg',
  '5LQq6DKdoh-5hRyVrR6BdhviqccyYO7I-iK_s488B3Y8RtNdiOBlBQUkd-hw-5CzHh_LJU1eoQp4Tg',
  'p5XGEgLDT77MrwEYqvKAnWcezJHzn7BvrldHu9LbYJw5yVyaVPblF8MDQXQMPglMBESL7uTk9XWhsA',
  'YuzBuj8BE28q8sCt-H4Zjmh1Z3ASgXN3Zr4vgt2zmZgdO-MkFoIBrOWIX1qvtmXEfWiVV5tLmeB84w',
  'jBFJb2VptK-A5krGaWXc2srhMnkx7vHgco8vs2vabkaXKQ9wPTLzxZdM7TH2CQUkBYyum17N-vsobQ',
  'YMbwqbdNhRFlAXpnE38kPHt5qdrqD5_clK0aJR-dsFCH7hElbD4H6XIRa6GmKYB63tT_BXt2STIFuA',
  'oRiPFVFS8450BfEkJ8x4DtsFHPdVbBiHJsbhC1qZ5I1cYMuPB4zXqF9rwqG6R2w5LX0n7dd2b2OeGw',
  'eYxF_0xQzhoxKZG4uIGOariy2Cr1arjGRDosg_7-PLtJ2QwrPVq3y_T6LG8UXfMN-yPRhwhdhD2bBg',
  'F1cPo_wDQAeppIrKnDD981FiFo9tYm5Ttkp3gQ8K4ydvHoC-ChLkCmEHtM5TBkdLXzVdFJU5HbkweA']}

In [6]:
#Look at the data at the beginning of the match
data['info']['frames'][0]

{'events': [{'realTimestamp': 1635396104102,
   'timestamp': 0,
   'type': 'PAUSE_END'}],
 'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
    'abilityPower': 0,
    'armor': 29,
    'armorPen': 0,
    'armorPenPercent': 0,
    'attackDamage': 25,
    'attackSpeed': 100,
    'bonusArmorPenPercent': 0,
    'bonusMagicPenPercent': 0,
    'ccReduction': 0,
    'cooldownReduction': 0,
    'health': 541,
    'healthMax': 541,
    'healthRegen': 0,
    'lifesteal': 0,
    'magicPen': 0,
    'magicPenPercent': 0,
    'magicResist': 30,
    'movementSpeed': 335,
    'omnivamp': 0,
    'physicalVamp': 0,
    'power': 200,
    'powerMax': 200,
    'powerRegen': 0,
    'spellVamp': 0},
   'currentGold': 500,
   'damageStats': {'magicDamageDone': 0,
    'magicDamageDoneToChampions': 0,
    'magicDamageTaken': 0,
    'physicalDamageDone': 0,
    'physicalDamageDoneToChampions': 0,
    'physicalDamageTaken': 0,
    'totalDamageDone': 0,
    'totalDamageDoneToChampions': 0,
    'tota

There are 2 sections to each index and each index appears to represent one minute of gameplay. The only exception is index 0, which is just a snapshot of the very beginning. 

The first section is 'events'. This section lists all of the events that occurred within that timeframe. Events are things like player kills, player deaths, level ups, etc. Joshua Chavez is working on turning all of the data in this section into a usable format.

The second section is 'particpantFrames'. This section is a dictionary of all players and their stats at the end of that timeframe. Stats are things like damage done, current gold, current level, total gold, etc. I will be working on this section and get all of the data in a usable format.

In [7]:
#Select the particpantFrames section for the very first timeframe
players = data['info']['frames'][0]['participantFrames']
players

{'1': {'championStats': {'abilityHaste': 0,
   'abilityPower': 0,
   'armor': 29,
   'armorPen': 0,
   'armorPenPercent': 0,
   'attackDamage': 25,
   'attackSpeed': 100,
   'bonusArmorPenPercent': 0,
   'bonusMagicPenPercent': 0,
   'ccReduction': 0,
   'cooldownReduction': 0,
   'health': 541,
   'healthMax': 541,
   'healthRegen': 0,
   'lifesteal': 0,
   'magicPen': 0,
   'magicPenPercent': 0,
   'magicResist': 30,
   'movementSpeed': 335,
   'omnivamp': 0,
   'physicalVamp': 0,
   'power': 200,
   'powerMax': 200,
   'powerRegen': 0,
   'spellVamp': 0},
  'currentGold': 500,
  'damageStats': {'magicDamageDone': 0,
   'magicDamageDoneToChampions': 0,
   'magicDamageTaken': 0,
   'physicalDamageDone': 0,
   'physicalDamageDoneToChampions': 0,
   'physicalDamageTaken': 0,
   'totalDamageDone': 0,
   'totalDamageDoneToChampions': 0,
   'totalDamageTaken': 0,
   'trueDamageDone': 0,
   'trueDamageDoneToChampions': 0,
   'trueDamageTaken': 0},
  'goldPerSecond': 0,
  'jungleMinionsKille

In [8]:
players['1']['currentGold']

500

In [9]:
#Now that I have the participantFrames, gather all the data and put it into a dict
#Will need to loop through each player.

players_dict = {}

for i in range(1, len(players) + 1):
    #Now create a temp dict to store the current players stats
    #Use formatted strings to automatically increment the player label
    temp_dict = {
        f'currentGold_{i}': players[str(i)]['currentGold'],
        f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
        f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
        f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
        f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
        f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
        f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
        f'level_{i}': players[str(i)]['level'],
        f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
        f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
        f'totalGold_{i}': players[str(i)]['totalGold'],
        f'xp_{i}': players[str(i)]['xp']
    }
    
    #Now that I have the current players stats, extend it to the overall players_dict
    players_dict.update(temp_dict)
    

In [10]:
players_dict

{'currentGold_1': 500,
 'magicDamageDoneToChampions_1': 0,
 'physicalDamageDoneToChampions_1': 0,
 'trueDamageDoneToChampions_1': 0,
 'totalDamageDoneToChampions_1': 0,
 'goldPerSecond_1': 0,
 'jungleMinionsKilled_1': 0,
 'level_1': 1,
 'minionsKilled_1': 0,
 'timeEnemySpentControlled_1': 0,
 'totalGold_1': 500,
 'xp_1': 0,
 'currentGold_2': 500,
 'magicDamageDoneToChampions_2': 0,
 'physicalDamageDoneToChampions_2': 0,
 'trueDamageDoneToChampions_2': 0,
 'totalDamageDoneToChampions_2': 0,
 'goldPerSecond_2': 0,
 'jungleMinionsKilled_2': 0,
 'level_2': 1,
 'minionsKilled_2': 0,
 'timeEnemySpentControlled_2': 0,
 'totalGold_2': 500,
 'xp_2': 0,
 'currentGold_3': 500,
 'magicDamageDoneToChampions_3': 0,
 'physicalDamageDoneToChampions_3': 0,
 'trueDamageDoneToChampions_3': 0,
 'totalDamageDoneToChampions_3': 0,
 'goldPerSecond_3': 0,
 'jungleMinionsKilled_3': 0,
 'level_3': 1,
 'minionsKilled_3': 0,
 'timeEnemySpentControlled_3': 0,
 'totalGold_3': 500,
 'xp_3': 0,
 'currentGold_4': 500,

In [11]:
#Now that I have been able to get the player stats in one time frame,
#Create a loop to get the rest

#I will need a list to store all the dictionaries from each timeframe
player_stats = []

#Here, each timeframe represents about one minute
for timeframe in data['info']['frames']:
    players = timeframe['participantFrames']
    
    #Now create a dicitonary to hold the players' stats from this timeframe
    players_dict = {}
    
    for i in range(1, len(players) + 1):
        #Now create a temp dict to store the current players stats
        #Use formatted strings to automatically increment the player label
        temp_dict = {
            f'currentGold_{i}': players[str(i)]['currentGold'],
            f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
            f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
            f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
            f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
            f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
            f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
            f'level_{i}': players[str(i)]['level'],
            f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
            f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
            f'totalGold_{i}': players[str(i)]['totalGold'],
            f'xp_{i}': players[str(i)]['xp']
        }

        #Now that I have the current players stats, extend it to the overall players_dict
        players_dict.update(temp_dict)
        
    #Update the players_dict one more time with the timestamp for the timeframe
    players_dict.update({'timestamp' : timeframe['timestamp']})
    
    #Append the players_dict to the overall player_stats list of dicts
    player_stats.append(players_dict)

In [12]:
#Now convert player_stats to a df
player_stats = pd.DataFrame(player_stats)

In [13]:
player_stats.head()

,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp
0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,0
1,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,20,0,1,0,0,500,0,50,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,20,0,1,0,0,500,0,60020
2,63,0,0,0,0,0,0,1,2,0,563,181,186,0,253,0,253,0,6,2,0,10836,686,449,112,0,78,0,78,0,0,1,5,3566,612,240,63,0,0,34,34,0,0,1,2,3933,563,75,44,0,138,0,138,20,0,1,1,4887,544,135,71,95,76,0,172,0,0,1,0,0,521,0,121,0,0,0,0,0,4,2,0,0,621,320,126,189,0,0,189,0,0,1,6,10470,626,270,98,0,0,0,0,0,0,1,4,0,598,199,43,73,0,0,73,20,0,1,0,0,543,18,120051
3,318,0,107,0,107,0,0,3,10,0,818,725,554,0,253,0,253,0,16,3,0,49180,1054,833,879,67,562,54,683,0,0,3,16,8258,1389,1018,632,0,438,110,549,0,0,2,10,20348,1132,393,336,0,619,90,709,20,0,2,3,8854,836,393,421,239,76,0,315,0,0,2,11,0,871,634,433,0,0,0,0,0,12,3,0,3897,933,685,395,209,39,0,248,0,0,3,14,38774,895,662,297,0,824,0,824,0,0,2,8,0,797,428,277,265,241,70,577,20,0,1,0,4459,777,187,180064
4,647,336,240,0,577,0,0,3,13,1486,1147,1109,1192,31,643,0,674,0,20,3,0,74848,1692,1085,688,172,1351,54,1577,0,0,4,17,12717,1948,1222,165,0,678,155,834,0,0,3,22,37257,1765,938,654,0,771,102,873,20,0,3,5,17377,1154,938,621,535,76,0,611,0,0,3,16,0,1071,994,876,0,676,88,764,0,12,3,0,28375,1376,823,783,356,84,0,44

Now, that I have all of the player stats from one match in this dataframe, I can simply access the index corresponding to the timeframe I want. Normally for this project, we would access the timeframe corresponding to the 20 minute mark, however this particular match didn't last that long.

In [14]:
len(player_stats)

19

In [15]:
player_stats.tail()

,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp
14,316,5347,1640,459,7448,0,0,10,73,43147,5776,6518,2318,146,2515,0,2661,0,87,9,6,260446,5893,6013,1968,426,3487,408,4321,0,1,10,95,51243,6578,6872,216,0,4054,419,4474,0,0,8,100,77230,5791,4725,260,0,2648,454,3103,30,0,7,23,51526,4160,4017,301,5630,168,366,6165,0,0,10,81,36466,5251,6496,289,0,3099,88,3187,0,48,8,11,74446,4539,4319,876,1416,127,0,1544,0,0,9,88,262222,3931,5446,1188,258,3383,0,3641,0,0,8,95,14801,4313,4167,651,1819,763,354,2937,30,0,7,2,18860,2926,3383,840308
15,271,5347,1640,459,7448,0,0,10,75,43147,6096,6919,2639,205,2720,0,2925,0,95,10,6,260512,6214,6521,630,510,4419,408,5338,0,1,10,95,52003,7140,7204,1401,0,6577,1020,7598,0,0,9,107,78650,6976,5367,1297,0,2794,644,3439,30,0,8,26,54036,5197,4816,696,5630,168,366,6165,0,0,10,93,41452,5646,7160,581,0,3564,164,3728,0,52,8,11,79103,4831,4824,1198,2210,127,0,2338,0,0,9,88,269190,4253,5946,310,285,5058,0,5344,0,0,8,95,18499,5035,4776,427,2166,917,392,3476,30,0,7,2,20710,3177,3603,900329
16,408,6456,1805,488,8750,0,0,10,76,44535,6233,7039,93,205,2720,0,2925,0,103,10,6,271114,6568,6992,997,568,5417,408,6394,0,7,11,101,57319,7507,7650,724,0,8206,1413,9619,0,0,9,107,81490,7399,5662,1479,0,2794,644,3439,30,0,9,28,56281,5379,5293,282,7364,168,366,7899,0,0,11,96,53379,6132,8021,799,0,3564,240,3804,0,56,9,11,94592,5049,5112,1657,2530,127,0,2658,0,0,10,104,290925,4712,6823,663,368,5876,101,6346,0,0,9,112,30523,5888,5646,797,2777,965,392,4135,30,0,8,2,25103,3547,4363,960345
17,658,6456,1805,488,8750,0,4,11,78,48633,6483,7446,585,205,2720,0,2925,0,107,11,16,306657,7060,8024,1148,659,6823,570,8052,0,7,11,103,61778,7658,79

In [16]:
player_stats.loc[18]

currentGold_1                         1505
magicDamageDoneToChampions_1          6877
physicalDamageDoneToChampions_1       1805
trueDamageDoneToChampions_1            519
totalDamageDoneToChampions_1          9202
                                    ...   
minionsKilled_10                         3
timeEnemySpentControlled_10          29330
totalGold_10                          4570
xp_10                                 4941
timestamp                          1066945
Name: 18, Length: 121, dtype: int64

# Gather Match IDs

Write a function to gather match IDs using puuids.

In [17]:
from time import sleep

#Create the function
def get_match_ids(puuids, api_key):
    """
    This function takes in a list of puuids and a riot games api key to gather
    and return a list of match ids.
    """
    
    #Create an empty list to store the match ids
    matches = []
    
    #Set the base url
    url = 'https://americas.api.riotgames.com/'
    
    #Set a counter var
    n = 0
    
    #Create another var to store the length of puuids
    num_puuids = len(puuids)
    
    #Loop through the puuids and gather 100 match ids from each of them
    for puuid in puuids:
        #Update counter
        n += 1

        #Build the query
        query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={0}&count={100}&api_key={api_key}'

        #Get the response
        response = requests.get(url + query)

        #Check response and leave a status message
        if response.status_code != 200:
            print(f'Something went wrong! Status Code {response.status_code} for puuid {puuid}. Skipping this puuid.\n')
            continue

        #Unpack the response as json
        #This should be a list of match ids
        data = response.json()

        #Extend the matches list with the data
        matches.extend(data)
        
        #Leave a status message after every 2 iterations (~6 minutes)
        if n % 2 == 0:
            print(f'Completed {n} of {num_puuids} puuids.')
            print(f'Total match IDs gathered: {len(matches)}\n')
        
        sleep(180)
        
    return matches

# Get My Own PUUIDs

In [18]:
def get_puuid(name_list,df,api_key):
    counter = 0
    for name in name_list:
        counter += 1
        if counter%50 == 0:
            print(counter)
            print('function paused')
            print
            sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
    return df

In [19]:
def automated_puuid(name_list,df,api_key):
    df = get_puuid(name_list,username_df,api_key)
    puuid_list = df.puuid
    matches = get_match_ids(puuid_list, api_key)
    return matches

In [20]:
#IMPORT 2640 players.csv
df = pd.read_csv('2640_players.csv')

In [21]:
# make the players name column into a series
player_list = df.players

In [22]:
# I will gather the values for 1200 - 1650
new_player_list = player_list[1200:1650]

In [23]:
# make an empty dataframe
username_df = pd.DataFrame(columns = ['puuid','username'])

In [24]:
# Get the match IDs using the automated_puuid function
#match_list = automated_puuid(new_player_list,username_df,api_key)

In [25]:
#len(match_list)

In [26]:
#Convert the list to a set to get rid of duplicates
#match_list = set(match_list)

In [27]:
#len(match_list)

In [28]:
import json

#Convert it to a df
#matches_df = pd.DataFrame(match_list)

In [29]:
#matches_df

In [30]:
#Save as csv file
#matches_df.to_csv('match_ids_smith.csv', index = False)

In [31]:
#Load the file for testing
matches_df = pd.read_csv('match_ids_smith.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
20313,NA1_4088936717
20314,NA1_4049964605
20315,NA1_4079861676
20316,NA1_4064742197


# Joining Match ID Files

Figure out how to join multiple csv files so that we have a complete list of match_ids.

In [32]:
chris_ids_1 = pd.read_csv('chris_match.csv', index_col = [0])
chris_ids_1

,0
0,NA1_4105161570
1,NA1_4102755048
2,NA1_4102698148
3,NA1_4102111080
4,NA1_4100394991
...,...
2958,NA1_4084901198
2959,NA1_4083884015
2960,NA1_4083778175
2961,NA1_4083754241


In [33]:
chris_ids_2 = pd.read_json('chris_match_pt_2.json')
chris_ids_2

,0
0,NA1_4103420054
1,NA1_4103325239
2,NA1_4103351973
3,NA1_4103268368
4,NA1_4103264369
...,...
5095,NA1_4082544943
5096,NA1_4082512507
5097,NA1_4082501374
5098,NA1_4082439390


In [34]:
bryant_ids = pd.read_csv('bryant_matches.csv', index_col = [0])
bryant_ids

,0
0,NA1_4105017995
1,NA1_4105001421
2,NA1_4104901232
3,NA1_4104795777
4,NA1_4104727661
...,...
26095,NA1_4088803193
26096,NA1_4088801150
26097,NA1_4088549060
26098,NA1_4088534067


In [35]:
#Drop duplicates from all
#matches_df.drop_duplicates(inplace = True)
#chris_ids_1.drop_duplicates(inplace = True)
#chris_ids_2.drop_duplicates(inplace = True)
#bryant_ids.drop_duplicates(inplace = True)

In [36]:
#Now join them all together
#matches_df = pd.concat([matches_df, chris_ids_1, chris_ids_2, bryant_ids], axis = 0, ignore_index = True)
#matches_df

In [37]:
#Drop the extra column
#matches_df = matches_df.drop(columns = [0])

In [38]:
#Drop duplicates one more time
#matches_df.drop_duplicates(inplace = True)
#matches_df

In [39]:
#Now save as a csv
#matches_df.to_csv('match_ids.csv', index = False)

In [40]:
#Reload for testing
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


# Getting Match Info

Build a function to gather the match json info and prepare it for exploration.

In [2]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [3]:
#Test the above function
import prepare

In [43]:
test_list = list(matches_df['0'])[:5]
test_list

['NA1_4062082172',
 'NA1_4093908534',
 'NA1_4094915650',
 'NA1_4082128299',
 'NA1_4083500908']

In [46]:
#Set the username
username = 'smith'

#Call the function
get_match_info(test_list, api_key, username)

Created timeline_data_smith.json
Created other_game_data_smith.json

Preparing the data...

Skipping: 0 due to <20 min
Finished with: 1 of 5
Finished with: 2 of 5
Finished with: 3 of 5
Finished with: 4 of 5


,killsplayer_1,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,killsplayer_10,deathsplayer_1,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,assistsplayer_2,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,assistsplayer_10,dragon_team100,dragon_team200,riftherald_team100,riftherald_team300,winningTeam,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp,gameMode,gameType,gameVersion,assistsplayer_1,assistsplayer_3,baron_team100,baron_team200,deathsplayer_10,killsplayer_0,riftherald_team200,killsplayer_5
0,5.0,1.0,2.0,2.0,3.0,5.0,9.0,3.0,1.0,4.0,4.0,6.0,4.0,3.0,5.0,2.0,2.0,1.0,2.0,1.0,4.0,1.0,7.0,5.0,2.0,9.0,1.0,3.0,1.0,1.0,200.0,93.0,0.0,9788.0,1446.0,11234.0,0.0,0.0,13.0,164.0,32908.0,8466.0,10075.0,449.0,265.0,3806.0,715.0,4786.0,0.0,116.0,11.0,10.0,174525.0,6874.0,7737.0,627.0,5296.0,426.0,408.0,6131.0,0.0,4.0,11.0,131.0,130689.0,6382.0,7833.0,933.0,445.0,4550.0,0.0,4996.0,0.0,0.0,11.0,169.0,49226.0,7433.0,7665.0,321.0,0.0,4275.0,0.0,4275.0,30.0,0.0,9.0,10.0,65677.0,4845.0,5191.0,1096.0,3339.0,1652.0,180.0,5172.0,0.0,11.0,12.0,139.0,192443.0,7171.0,9935.0,153.0,1214.0,8330.0,436.0,9980.0,0.0,98.0,12.0,35.0,439876.0,8243.0,9736.0,272.0,11641.0,2292.0,2184.0,16117.0,0.0,4.0,12.0,132.0,112457.0,8697.0,9829.0,93.0,0.0,3707.0,519.0,4227.0,0.0,24.0,11.0,141.0,6375.0,7643.0,8362.0,176.0,5672.0,625.0,138.0,6436.0,30.0,0.0,10.0,14.0,32181.0,5611.0,7173.0,1200345.0,CLASSIC,MATCHED_GAME,11.22.406.3587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,6.0,7.0,1.0,NaN,4.0,2.0,3.0

In [10]:
#Now try loading the json files that were generated
timeline_data = pd.read_json('../timeline_data_smith.json')
game_data = pd.read_json('../other_game_data_smith.json')

In [12]:
#Will need to change it to a dictionary to access it like normal
timeline_list = timeline_data.to_dict(orient = 'records')
game_data_list = game_data.to_dict(orient = 'records')

In [15]:
import prepare

In [16]:
#Now feed these lists into the prepare function
df = prepare.prepare(timeline_list, game_data_list, 20)

Skipping: 0 due to <20 min or not classic
Finished with: 1 of 1000
Finished with: 2 of 1000
Finished with: 3 of 1000
Finished with: 4 of 1000
Finished with: 5 of 1000
Finished with: 6 of 1000
Finished with: 7 of 1000
Finished with: 8 of 1000
Finished with: 9 of 1000
Finished with: 10 of 1000
Finished with: 11 of 1000
Finished with: 12 of 1000
Finished with: 13 of 1000
Finished with: 14 of 1000
Finished with: 15 of 1000
Finished with: 16 of 1000
Finished with: 17 of 1000
Finished with: 18 of 1000
Skipping: 19 due to <20 min or not classic
Finished with: 20 of 1000
Skipping: 21 due to <20 min or not classic
Finished with: 22 of 1000
Finished with: 23 of 1000
Skipping: 24 due to <20 min or not classic
Finished with: 25 of 1000
Finished with: 26 of 1000
Finished with: 27 of 1000
Finished with: 28 of 1000
Finished with: 29 of 1000
Finished with: 30 of 1000
Skipping: 31 due to <20 min or not classic
Skipping: 32 due to <20 min or not classic
Skipping: 33 due to <20 min or not classic
Skippin

Finished with: 264 of 1000
Finished with: 265 of 1000
Finished with: 266 of 1000
Finished with: 267 of 1000
Finished with: 268 of 1000
Finished with: 269 of 1000
Finished with: 270 of 1000
Finished with: 271 of 1000
Finished with: 272 of 1000
Skipping: 273 due to <20 min or not classic
Skipping: 274 due to <20 min or not classic
Skipping: 275 due to <20 min or not classic
Finished with: 276 of 1000
Skipping: 277 due to <20 min or not classic
Finished with: 278 of 1000
Skipping: 279 due to <20 min or not classic
Finished with: 280 of 1000
Finished with: 281 of 1000
Finished with: 282 of 1000
Finished with: 283 of 1000
Finished with: 284 of 1000
Finished with: 285 of 1000
Finished with: 286 of 1000
Skipping: 287 due to <20 min or not classic
Skipping: 288 due to <20 min or not classic
Finished with: 289 of 1000
Skipping: 290 due to <20 min or not classic
Finished with: 291 of 1000
Finished with: 292 of 1000
Finished with: 293 of 1000
Skipping: 294 due to <20 min or not classic
Skipping: 

Finished with: 521 of 1000
Finished with: 522 of 1000
Skipping: 523 due to <20 min or not classic
Finished with: 524 of 1000
Finished with: 525 of 1000
Finished with: 526 of 1000
Finished with: 527 of 1000
Finished with: 528 of 1000
Finished with: 529 of 1000
Finished with: 530 of 1000
Finished with: 531 of 1000
Finished with: 532 of 1000
Skipping: 533 due to <20 min or not classic
Finished with: 534 of 1000
Finished with: 535 of 1000
Skipping: 536 due to <20 min or not classic
Skipping: 537 due to <20 min or not classic
Finished with: 538 of 1000
Skipping: 539 due to <20 min or not classic
Finished with: 540 of 1000
Finished with: 541 of 1000
Finished with: 542 of 1000
Finished with: 543 of 1000
Finished with: 544 of 1000
Skipping: 545 due to <20 min or not classic
Skipping: 546 due to <20 min or not classic
Finished with: 547 of 1000
Finished with: 548 of 1000
Finished with: 549 of 1000
Finished with: 550 of 1000
Finished with: 551 of 1000
Finished with: 552 of 1000
Finished with: 55

Finished with: 780 of 1000
Skipping: 781 due to <20 min or not classic
Finished with: 782 of 1000
Finished with: 783 of 1000
Finished with: 784 of 1000
Skipping: 785 due to <20 min or not classic
Finished with: 786 of 1000
Finished with: 787 of 1000
Skipping: 788 due to <20 min or not classic
Finished with: 789 of 1000
Skipping: 790 due to <20 min or not classic
Skipping: 791 due to <20 min or not classic
Finished with: 792 of 1000
Finished with: 793 of 1000
Finished with: 794 of 1000
Skipping: 795 due to <20 min or not classic
Skipping: 796 due to <20 min or not classic
Finished with: 797 of 1000
Finished with: 798 of 1000
Skipping: 799 due to <20 min or not classic
Finished with: 800 of 1000
Finished with: 801 of 1000
Skipping: 802 due to <20 min or not classic
Skipping: 803 due to <20 min or not classic
Finished with: 804 of 1000
Finished with: 805 of 1000
Skipping: 806 due to <20 min or not classic
Skipping: 807 due to <20 min or not classic
Finished with: 808 of 1000
Finished with

In [18]:
#Now save this df 
df.to_csv('match_data_start_0_end_1000.csv', index = False)

# Gather My Portion of the Data

In [3]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


In [4]:
#Set up the match id list with a thousand entries
match_ids = list(matches_df['0'])[4000:10_657]

In [6]:
from time import sleep

In [7]:
#Before calling the get_match_info function, separate the match_ids list into smaller batches
match_batch_1 = match_ids[:1000]
match_batch_2 = match_ids[1000: 2000]
match_batch_3 = match_ids[2000: 3000]
match_batch_4 = match_ids[3000: 4000]
match_batch_5 = match_ids[4000: 5000]
match_batch_6 = match_ids[5000: 6000]
match_batch_7 = match_ids[6000: 6657]

In [21]:
#Since I will be running this function in smaller batches, I will have to update the username
#after each iteration. This will allow me to keep track of which json files contain what data

#For first iteration
username = 'start_4000_end_5000'

#Call the get_match_info function
df1 = get_match_info(match_batch_1, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 216 of 1000
Skipping: 217 due to <20 min or not classic
Finished with: 218 of 1000
Finished with: 219 of 1000
Finished with: 220 of 1000
Finished with: 221 of 1000
Finished with: 222 of 1000
Finished with: 223 of 1000
Skipping: 224 due to <20 min or not classic
Skipping: 225 due to <20 min or not classic
Finished with: 226 of 1000
Finished with: 227 of 1000
Skipping: 228 due to <20 min or not classic
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Skipping: 232 due to <20 min or not classic
Finished with: 233 of 1000
Finished with: 234 of 1000
Finished with: 235 of 1000
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Finished with: 239 of 1000
Finished with: 240 of 1000
Skipping: 241 due to <20 min or not classic
Finished with: 242 of 1000
Finished with: 243 of 1000
Finished with: 244 of 1000
Finished with: 245 of 1000
Finished with: 246 of 1000
Finished with: 247 of 1000
Finished with: 248 of 1000
Finishe

Finished with: 473 of 1000
Skipping: 474 due to <20 min or not classic
Finished with: 475 of 1000
Finished with: 476 of 1000
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Skipping: 480 due to <20 min or not classic
Finished with: 481 of 1000
Finished with: 482 of 1000
Finished with: 483 of 1000
Finished with: 484 of 1000
Finished with: 485 of 1000
Skipping: 486 due to <20 min or not classic
Finished with: 487 of 1000
Finished with: 488 of 1000
Finished with: 489 of 1000
Skipping: 490 due to <20 min or not classic
Finished with: 491 of 1000
Finished with: 492 of 1000
Skipping: 493 due to <20 min or not classic
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 of 1000
Finished with: 497 of 1000
Finished with: 498 of 1000
Finished with: 499 of 1000
Skipping: 500 due to <20 min or not classic
Finished with: 501 of 1000
Finished with: 502 of 1000
Skipping: 503 due to <20 min or not classic
Finished with: 504 of 1000
Skipping: 505 due

Finished with: 730 of 1000
Skipping: 731 due to <20 min or not classic
Finished with: 732 of 1000
Finished with: 733 of 1000
Finished with: 734 of 1000
Finished with: 735 of 1000
Finished with: 736 of 1000
Finished with: 737 of 1000
Finished with: 738 of 1000
Finished with: 739 of 1000
Finished with: 740 of 1000
Skipping: 741 due to <20 min or not classic
Finished with: 742 of 1000
Skipping: 743 due to <20 min or not classic
Finished with: 744 of 1000
Finished with: 745 of 1000
Finished with: 746 of 1000
Finished with: 747 of 1000
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Skipping: 752 due to <20 min or not classic
Skipping: 753 due to <20 min or not classic
Skipping: 754 due to <20 min or not classic
Finished with: 755 of 1000
Finished with: 756 of 1000
Finished with: 757 of 1000
Skipping: 758 due to <20 min or not classic
Finished with: 759 of 1000
Finished with: 760 of 1000
Skipping: 761 due to <20 min or not classic


Finished with: 984 of 1000
Skipping: 985 due to <20 min or not classic
Finished with: 986 of 1000
Skipping: 987 due to <20 min or not classic
Finished with: 988 of 1000
Finished with: 989 of 1000
Skipping: 990 due to <20 min or not classic
Skipping: 991 due to <20 min or not classic
Finished with: 992 of 1000
Finished with: 993 of 1000
Finished with: 994 of 1000
Finished with: 995 of 1000
Finished with: 996 of 1000
Finished with: 997 of 1000
Skipping: 998 due to <20 min or not classic
Skipping: 999 due to <20 min or not classic


In [22]:
#Now save this df 
df1.to_csv(f'match_data_{username}.csv', index = False)

In [23]:
#For second iteration
username = 'start_5000_end_6000'

#Call the get_match_info function
df2 = get_match_info(match_batch_2, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Something went wrong getting TIMELINE DATA! Status Code 503 for match ID: NA1_4091906261.
Skipping this match ID.

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 s

Finished with: 213 of 999
Skipping: 214 due to <20 min or not classic
Finished with: 215 of 999
Finished with: 216 of 999
Finished with: 217 of 999
Skipping: 218 due to <20 min or not classic
Finished with: 219 of 999
Finished with: 220 of 999
Finished with: 221 of 999
Finished with: 222 of 999
Finished with: 223 of 999
Finished with: 224 of 999
Finished with: 225 of 999
Skipping: 226 due to <20 min or not classic
Skipping: 227 due to <20 min or not classic
Skipping: 228 due to <20 min or not classic
Finished with: 229 of 999
Finished with: 230 of 999
Skipping: 231 due to <20 min or not classic
Finished with: 232 of 999
Finished with: 233 of 999
Finished with: 234 of 999
Skipping: 235 due to <20 min or not classic
Skipping: 236 due to <20 min or not classic
Finished with: 237 of 999
Finished with: 238 of 999
Finished with: 239 of 999
Finished with: 240 of 999
Finished with: 241 of 999
Skipping: 242 due to <20 min or not classic
Finished with: 243 of 999
Finished with: 244 of 999
Finish

Finished with: 483 of 999
Skipping: 484 due to <20 min or not classic
Skipping: 485 due to <20 min or not classic
Finished with: 486 of 999
Finished with: 487 of 999
Finished with: 488 of 999
Finished with: 489 of 999
Finished with: 490 of 999
Finished with: 491 of 999
Finished with: 492 of 999
Finished with: 493 of 999
Finished with: 494 of 999
Finished with: 495 of 999
Finished with: 496 of 999
Skipping: 497 due to <20 min or not classic
Finished with: 498 of 999
Finished with: 499 of 999
Finished with: 500 of 999
Skipping: 501 due to <20 min or not classic
Finished with: 502 of 999
Skipping: 503 due to <20 min or not classic
Finished with: 504 of 999
Finished with: 505 of 999
Skipping: 506 due to <20 min or not classic
Finished with: 507 of 999
Finished with: 508 of 999
Finished with: 509 of 999
Finished with: 510 of 999
Finished with: 511 of 999
Skipping: 512 due to <20 min or not classic
Finished with: 513 of 999
Skipping: 514 due to <20 min or not classic
Skipping: 515 due to <20

Finished with: 748 of 999
Skipping: 749 due to <20 min or not classic
Skipping: 750 due to <20 min or not classic
Finished with: 751 of 999
Finished with: 752 of 999
Finished with: 753 of 999
Finished with: 754 of 999
Skipping: 755 due to <20 min or not classic
Finished with: 756 of 999
Skipping: 757 due to <20 min or not classic
Skipping: 758 due to <20 min or not classic
Finished with: 759 of 999
Finished with: 760 of 999
Finished with: 761 of 999
Skipping: 762 due to <20 min or not classic
Finished with: 763 of 999
Finished with: 764 of 999
Skipping: 765 due to <20 min or not classic
Skipping: 766 due to <20 min or not classic
Finished with: 767 of 999
Skipping: 768 due to <20 min or not classic
Skipping: 769 due to <20 min or not classic
Finished with: 770 of 999
Finished with: 771 of 999
Finished with: 772 of 999
Finished with: 773 of 999
Finished with: 774 of 999
Finished with: 775 of 999
Finished with: 776 of 999
Skipping: 777 due to <20 min or not classic
Finished with: 778 of 

In [24]:
#Now save this df 
df2.to_csv(f'match_data_{username}.csv', index = False)

In [25]:
#For third iteration
username = 'start_6000_end_7000'

#Call the get_match_info function
df3 = get_match_info(match_batch_3, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Something went wrong getting OTHER GAME DATA! Status Code 503 for match ID: NA1_4060916551.
Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.

Something went wrong getting TIMELINE DATA! Status Code 503 for match ID: NA1_4082883633.
Skipping this match ID.

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.

Finished with: 206 of 998
Finished with: 207 of 998
Finished with: 208 of 998
Finished with: 209 of 998
Skipping: 210 due to <20 min or not classic
Finished with: 211 of 998
Finished with: 212 of 998
Skipping: 213 due to <20 min or not classic
Finished with: 214 of 998
Finished with: 215 of 998
Finished with: 216 of 998
Finished with: 217 of 998
Finished with: 218 of 998
Finished with: 219 of 998
Finished with: 220 of 998
Finished with: 221 of 998
Finished with: 222 of 998
Finished with: 223 of 998
Finished with: 224 of 998
Finished with: 225 of 998
Skipping: 226 due to <20 min or not classic
Finished with: 227 of 998
Skipping: 228 due to <20 min or not classic
Finished with: 229 of 998
Finished with: 230 of 998
Finished with: 231 of 998
Finished with: 232 of 998
Finished with: 233 of 998
Skipping: 234 due to <20 min or not classic
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 998
Skipping: 237 due to <20 min or not classic
Skipping: 238 due to <20 min or not classi

Finished with: 467 of 998
Finished with: 468 of 998
Finished with: 469 of 998
Finished with: 470 of 998
Finished with: 471 of 998
Finished with: 472 of 998
Finished with: 473 of 998
Finished with: 474 of 998
Finished with: 475 of 998
Skipping: 476 due to <20 min or not classic
Finished with: 477 of 998
Finished with: 478 of 998
Finished with: 479 of 998
Finished with: 480 of 998
Finished with: 481 of 998
Skipping: 482 due to <20 min or not classic
Finished with: 483 of 998
Finished with: 484 of 998
Finished with: 485 of 998
Finished with: 486 of 998
Skipping: 487 due to <20 min or not classic
Finished with: 488 of 998
Skipping: 489 due to <20 min or not classic
Finished with: 490 of 998
Finished with: 491 of 998
Skipping: 492 due to <20 min or not classic
Finished with: 493 of 998
Finished with: 494 of 998
Finished with: 495 of 998
Skipping: 496 due to <20 min or not classic
Finished with: 497 of 998
Finished with: 498 of 998
Finished with: 499 of 998
Finished with: 500 of 998
Finished

Finished with: 732 of 998
Finished with: 733 of 998
Skipping: 734 due to <20 min or not classic
Finished with: 735 of 998
Skipping: 736 due to <20 min or not classic
Finished with: 737 of 998
Finished with: 738 of 998
Skipping: 739 due to <20 min or not classic
Finished with: 740 of 998
Skipping: 741 due to <20 min or not classic
Skipping: 742 due to <20 min or not classic
Finished with: 743 of 998
Finished with: 744 of 998
Finished with: 745 of 998
Finished with: 746 of 998
Finished with: 747 of 998
Skipping: 748 due to <20 min or not classic
Finished with: 749 of 998
Finished with: 750 of 998
Finished with: 751 of 998
Finished with: 752 of 998
Skipping: 753 due to <20 min or not classic
Finished with: 754 of 998
Finished with: 755 of 998
Skipping: 756 due to <20 min or not classic
Skipping: 757 due to <20 min or not classic
Finished with: 758 of 998
Finished with: 759 of 998
Finished with: 760 of 998
Skipping: 761 due to <20 min or not classic
Skipping: 762 due to <20 min or not clas

In [26]:
#Now save this df 
df3.to_csv(f'match_data_{username}.csv', index = False)

In [27]:
#For fourth iteration
username = 'start_7000_end_8000'

#Call the get_match_info function
df4 = get_match_info(match_batch_4, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 217 of 1000
Skipping: 218 due to <20 min or not classic
Finished with: 219 of 1000
Finished with: 220 of 1000
Skipping: 221 due to <20 min or not classic
Skipping: 222 due to <20 min or not classic
Finished with: 223 of 1000
Skipping: 224 due to <20 min or not classic
Finished with: 225 of 1000
Skipping: 226 due to <20 min or not classic
Finished with: 227 of 1000
Skipping: 228 due to <20 min or not classic
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Finished with: 232 of 1000
Skipping: 233 due to <20 min or not classic
Finished with: 234 of 1000
Finished with: 235 of 1000
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Finished with: 239 of 1000
Finished with: 240 of 1000
Finished with: 241 of 1000
Skipping: 242 due to <20 min or not classic
Finished with: 243 of 1000
Skipping: 244 due to <20 min or not classic
Finished with: 245 of 1000
Finished with: 246 of 1000
Skipping: 247 due to <20 min or not c

Finished with: 475 of 1000
Finished with: 476 of 1000
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Skipping: 480 due to <20 min or not classic
Finished with: 481 of 1000
Finished with: 482 of 1000
Finished with: 483 of 1000
Skipping: 484 due to <20 min or not classic
Finished with: 485 of 1000
Skipping: 486 due to <20 min or not classic
Finished with: 487 of 1000
Finished with: 488 of 1000
Finished with: 489 of 1000
Finished with: 490 of 1000
Skipping: 491 due to <20 min or not classic
Finished with: 492 of 1000
Finished with: 493 of 1000
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 of 1000
Finished with: 497 of 1000
Finished with: 498 of 1000
Finished with: 499 of 1000
Skipping: 500 due to <20 min or not classic
Finished with: 501 of 1000
Finished with: 502 of 1000
Skipping: 503 due to <20 min or not classic
Finished with: 504 of 1000
Skipping: 505 due to <20 min or not classic
Finished with: 506 of 1000
Skipping: 507 due

Finished with: 735 of 1000
Finished with: 736 of 1000
Finished with: 737 of 1000
Finished with: 738 of 1000
Skipping: 739 due to <20 min or not classic
Finished with: 740 of 1000
Skipping: 741 due to <20 min or not classic
Finished with: 742 of 1000
Finished with: 743 of 1000
Finished with: 744 of 1000
Skipping: 745 due to <20 min or not classic
Finished with: 746 of 1000
Finished with: 747 of 1000
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Skipping: 752 due to <20 min or not classic
Finished with: 753 of 1000
Skipping: 754 due to <20 min or not classic
Skipping: 755 due to <20 min or not classic
Finished with: 756 of 1000
Finished with: 757 of 1000
Finished with: 758 of 1000
Finished with: 759 of 1000
Finished with: 760 of 1000
Finished with: 761 of 1000
Finished with: 762 of 1000
Skipping: 763 due to <20 min or not classic
Finished with: 764 of 1000
Finished with: 765 of 1000
Finished with: 766 of 1000
Finished with: 76

Finished with: 991 of 1000
Finished with: 992 of 1000
Finished with: 993 of 1000
Finished with: 994 of 1000
Finished with: 995 of 1000
Skipping: 996 due to <20 min or not classic
Finished with: 997 of 1000
Finished with: 998 of 1000
Finished with: 999 of 1000


In [28]:
#Now save this df 
df4.to_csv(f'match_data_{username}.csv', index = False)

In [29]:
#For fifth iteration
username = 'start_8000_end_9000'

#Call the get_match_info function
df5 = get_match_info(match_batch_5, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 213 of 1000
Finished with: 214 of 1000
Finished with: 215 of 1000
Finished with: 216 of 1000
Finished with: 217 of 1000
Finished with: 218 of 1000
Finished with: 219 of 1000
Finished with: 220 of 1000
Finished with: 221 of 1000
Finished with: 222 of 1000
Finished with: 223 of 1000
Skipping: 224 due to <20 min or not classic
Finished with: 225 of 1000
Finished with: 226 of 1000
Finished with: 227 of 1000
Skipping: 228 due to <20 min or not classic
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Skipping: 232 due to <20 min or not classic
Finished with: 233 of 1000
Skipping: 234 due to <20 min or not classic
Finished with: 235 of 1000
Skipping: 236 due to <20 min or not classic
Skipping: 237 due to <20 min or not classic
Finished with: 238 of 1000
Finished with: 239 of 1000
Finished with: 240 of 1000
Finished with: 241 of 1000
Skipping: 242 due to <20 min or not classic
Finished with: 243 of 1000
Skipping: 244 due to <20 min or not classic


Finished with: 473 of 1000
Skipping: 474 due to <20 min or not classic
Finished with: 475 of 1000
Finished with: 476 of 1000
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Finished with: 480 of 1000
Skipping: 481 due to <20 min or not classic
Skipping: 482 due to <20 min or not classic
Skipping: 483 due to <20 min or not classic
Skipping: 484 due to <20 min or not classic
Skipping: 485 due to <20 min or not classic
Finished with: 486 of 1000
Finished with: 487 of 1000
Finished with: 488 of 1000
Finished with: 489 of 1000
Finished with: 490 of 1000
Finished with: 491 of 1000
Finished with: 492 of 1000
Skipping: 493 due to <20 min or not classic
Skipping: 494 due to <20 min or not classic
Finished with: 495 of 1000
Finished with: 496 of 1000
Skipping: 497 due to <20 min or not classic
Skipping: 498 due to <20 min or not classic
Finished with: 499 of 1000
Skipping: 500 due to <20 min or not classic
Finished with: 501 of 1000
Finished with: 502 of 1000
Ski

Finished with: 728 of 1000
Skipping: 729 due to <20 min or not classic
Finished with: 730 of 1000
Skipping: 731 due to <20 min or not classic
Finished with: 732 of 1000
Finished with: 733 of 1000
Finished with: 734 of 1000
Skipping: 735 due to <20 min or not classic
Finished with: 736 of 1000
Finished with: 737 of 1000
Finished with: 738 of 1000
Skipping: 739 due to <20 min or not classic
Finished with: 740 of 1000
Finished with: 741 of 1000
Finished with: 742 of 1000
Skipping: 743 due to <20 min or not classic
Skipping: 744 due to <20 min or not classic
Finished with: 745 of 1000
Finished with: 746 of 1000
Finished with: 747 of 1000
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Finished with: 752 of 1000
Finished with: 753 of 1000
Finished with: 754 of 1000
Finished with: 755 of 1000
Finished with: 756 of 1000
Finished with: 757 of 1000
Finished with: 758 of 1000
Finished with: 759 of 1000
Finished with: 760 of 1000
Skippin

Finished with: 983 of 1000
Finished with: 984 of 1000
Finished with: 985 of 1000
Skipping: 986 due to <20 min or not classic
Finished with: 987 of 1000
Finished with: 988 of 1000
Finished with: 989 of 1000
Finished with: 990 of 1000
Skipping: 991 due to <20 min or not classic
Skipping: 992 due to <20 min or not classic
Skipping: 993 due to <20 min or not classic
Skipping: 994 due to <20 min or not classic
Finished with: 995 of 1000
Skipping: 996 due to <20 min or not classic
Finished with: 997 of 1000
Finished with: 998 of 1000
Finished with: 999 of 1000


In [30]:
#Now save this df 
df5.to_csv(f'match_data_{username}.csv', index = False)

In [31]:
#For sixth iteration
username = 'start_9000_end_10000'

#Call the get_match_info function
df6 = get_match_info(match_batch_6, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 212 of 1000
Finished with: 213 of 1000
Finished with: 214 of 1000
Finished with: 215 of 1000
Finished with: 216 of 1000
Finished with: 217 of 1000
Finished with: 218 of 1000
Skipping: 219 due to <20 min or not classic
Finished with: 220 of 1000
Finished with: 221 of 1000
Skipping: 222 due to <20 min or not classic
Skipping: 223 due to <20 min or not classic
Skipping: 224 due to <20 min or not classic
Finished with: 225 of 1000
Finished with: 226 of 1000
Finished with: 227 of 1000
Finished with: 228 of 1000
Finished with: 229 of 1000
Skipping: 230 due to <20 min or not classic
Finished with: 231 of 1000
Finished with: 232 of 1000
Finished with: 233 of 1000
Finished with: 234 of 1000
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Finished with: 239 of 1000
Finished with: 240 of 1000
Finished with: 241 of 1000
Finished with: 242 of 1000
Finished with: 243 of 1000
Finished with: 244 of 1000
Skippin

Finished with: 466 of 1000
Skipping: 467 due to <20 min or not classic
Finished with: 468 of 1000
Finished with: 469 of 1000
Finished with: 470 of 1000
Skipping: 471 due to <20 min or not classic
Finished with: 472 of 1000
Skipping: 473 due to <20 min or not classic
Skipping: 474 due to <20 min or not classic
Finished with: 475 of 1000
Skipping: 476 due to <20 min or not classic
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Finished with: 480 of 1000
Finished with: 481 of 1000
Finished with: 482 of 1000
Finished with: 483 of 1000
Skipping: 484 due to <20 min or not classic
Finished with: 485 of 1000
Finished with: 486 of 1000
Finished with: 487 of 1000
Skipping: 488 due to <20 min or not classic
Finished with: 489 of 1000
Skipping: 490 due to <20 min or not classic
Skipping: 491 due to <20 min or not classic
Skipping: 492 due to <20 min or not classic
Finished with: 493 of 1000
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 o

Finished with: 730 of 1000
Finished with: 731 of 1000
Finished with: 732 of 1000
Skipping: 733 due to <20 min or not classic
Skipping: 734 due to <20 min or not classic
Finished with: 735 of 1000
Finished with: 736 of 1000
Finished with: 737 of 1000
Finished with: 738 of 1000
Finished with: 739 of 1000
Finished with: 740 of 1000
Finished with: 741 of 1000
Finished with: 742 of 1000
Finished with: 743 of 1000
Finished with: 744 of 1000
Skipping: 745 due to <20 min or not classic
Skipping: 746 due to <20 min or not classic
Finished with: 747 of 1000
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Finished with: 752 of 1000
Finished with: 753 of 1000
Finished with: 754 of 1000
Finished with: 755 of 1000
Skipping: 756 due to <20 min or not classic
Skipping: 757 due to <20 min or not classic
Skipping: 758 due to <20 min or not classic
Skipping: 759 due to <20 min or not classic
Finished with: 760 of 1000
Finished with: 761 of 1000


Finished with: 991 of 1000
Finished with: 992 of 1000
Skipping: 993 due to <20 min or not classic
Skipping: 994 due to <20 min or not classic
Finished with: 995 of 1000
Finished with: 996 of 1000
Skipping: 997 due to <20 min or not classic
Finished with: 998 of 1000
Skipping: 999 due to <20 min or not classic


In [32]:
#Now save this df 
df6.to_csv(f'match_data_{username}.csv', index = False)

In [33]:
#Final iteration
username = 'start_10000_end_10657'

#Call the get_match_info function
df7 = get_match_info(match_batch_7, api_key, username)

Completed 50 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Created timeline_data_start_10000_end_10657.json
Created other_game_data_start_10000_end_10657.json

Pre

Finished with: 238 of 657
Finished with: 239 of 657
Finished with: 240 of 657
Finished with: 241 of 657
Finished with: 242 of 657
Finished with: 243 of 657
Finished with: 244 of 657
Finished with: 245 of 657
Skipping: 246 due to <20 min or not classic
Finished with: 247 of 657
Finished with: 248 of 657
Finished with: 249 of 657
Skipping: 250 due to <20 min or not classic
Finished with: 251 of 657
Finished with: 252 of 657
Finished with: 253 of 657
Finished with: 254 of 657
Finished with: 255 of 657
Finished with: 256 of 657
Finished with: 257 of 657
Finished with: 258 of 657
Finished with: 259 of 657
Finished with: 260 of 657
Finished with: 261 of 657
Finished with: 262 of 657
Finished with: 263 of 657
Skipping: 264 due to <20 min or not classic
Finished with: 265 of 657
Finished with: 266 of 657
Finished with: 267 of 657
Finished with: 268 of 657
Finished with: 269 of 657
Skipping: 270 due to <20 min or not classic
Finished with: 271 of 657
Finished with: 272 of 657
Finished with: 273

Finished with: 504 of 657
Skipping: 505 due to <20 min or not classic
Finished with: 506 of 657
Finished with: 507 of 657
Finished with: 508 of 657
Skipping: 509 due to <20 min or not classic
Skipping: 510 due to <20 min or not classic
Finished with: 511 of 657
Skipping: 512 due to <20 min or not classic
Skipping: 513 due to <20 min or not classic
Finished with: 514 of 657
Finished with: 515 of 657
Skipping: 516 due to <20 min or not classic
Finished with: 517 of 657
Finished with: 518 of 657
Finished with: 519 of 657
Skipping: 520 due to <20 min or not classic
Finished with: 521 of 657
Finished with: 522 of 657
Finished with: 523 of 657
Skipping: 524 due to <20 min or not classic
Skipping: 525 due to <20 min or not classic
Skipping: 526 due to <20 min or not classic
Finished with: 527 of 657
Finished with: 528 of 657
Finished with: 529 of 657
Finished with: 530 of 657
Finished with: 531 of 657
Finished with: 532 of 657
Skipping: 533 due to <20 min or not classic
Finished with: 534 of 

In [34]:
#Now save this df 
df7.to_csv(f'match_data_{username}.csv', index = False)

__Now Combine DataFrames__

The data starting at 3000 and ending at 5000 has already been added to the final.csv in the github repo. Just concat the remaining dataframes and push that.

In [38]:
large_df = df2.append([df3, df4, df5, df6, df7], ignore_index = True)
large_df

,killsplayer_1,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,killsplayer_10,deathsplayer_1,deathsplayer_2,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,deathsplayer_10,assistsplayer_1,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,assistsplayer_10,dragon_team100,dragon_team200,riftherald_team200,riftherald_team300,baron_team100,winningTeam,ward_player_1,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,ward_player_10,towers_lost_team100,towers_lost_team200,inhibs_lost_team200,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp,gameMode,gameType,gameVersion,deathsplayer_3,inhibs_lost_team100,riftherald_team100,killsplayer_0,baron_team200,ward_player_0,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

In [40]:
#Are there really only 4037 rows left?
len(df2) + len(df3) + len(df4) + len(df5) + len(df6) + len(df7)

4037

In [39]:
#Now save df and push it to github
large_df.to_csv('match_data_start_5000_end_10657.csv', index = False)

In [42]:
#Now update the final.csv file
#Load it
final_df = pd.read_csv('../final.csv', index_col = [0])
final_df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,baron_team100,baron_team200,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_10,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,dragon_team100,dragon_team200,gameMode,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team200,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_1,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_7,killsplayer_8,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,riftherald_team200,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,inhibs_lost_team100,killsplayer_6,killsplayer_9,riftherald_team100,ward_player_0,killsplayer_0,riftherald_team300,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

In [45]:
final_df.isna().sum()

assistsplayer_1              0
assistsplayer_10             0
assistsplayer_2              0
assistsplayer_3              0
assistsplayer_4              0
                          ... 
team_assistsplayer_100       0
team_assistsplayer_200       0
team_xp_100                  0
team_xp_200                  0
Unnamed: 0.1              3922
Length: 209, dtype: int64

In [46]:
final_df['Unnamed: 0.1'].value_counts()

0.0       1
742.0     1
732.0     1
733.0     1
734.0     1
         ..
469.0     1
470.0     1
471.0     1
472.0     1
1016.0    1
Name: Unnamed: 0.1, Length: 814, dtype: int64

In [47]:
#Drop the unnamed column.
final_df.drop(columns = ['Unnamed: 0.1'], inplace = True)

In [48]:
final_df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,baron_team100,baron_team200,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_10,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,dragon_team100,dragon_team200,gameMode,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team200,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_1,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_7,killsplayer_8,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,riftherald_team200,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,inhibs_lost_team100,killsplayer_6,killsplayer_9,riftherald_team100,ward_player_0,killsplayer_0,riftherald_team300,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

In [49]:
#Now join this dataframe with your large_df
final_df = final_df.append(large_df, ignore_index = True)

In [50]:
final_df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,baron_team100,baron_team200,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_10,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,dragon_team100,dragon_team200,gameMode,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team200,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_1,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_7,killsplayer_8,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,riftherald_team200,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,inhibs_lost_team100,killsplayer_6,killsplayer_9,riftherald_team100,ward_player_0,killsplayer_0,riftherald_team300,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

In [51]:
#Now save it as the new final.csv
final_df.to_csv('final.csv', index = False)

In [52]:
#Now load it again to verify it worked properly
final_df = pd.read_csv('../final.csv')
final_df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,baron_team100,baron_team200,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_10,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_7,deathsplayer_8,deathsplayer_9,dragon_team100,dragon_team200,gameMode,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team200,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_1,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_7,killsplayer_8,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,riftherald_team200,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,inhibs_lost_team100,killsplayer_6,killsplayer_9,riftherald_team100,ward_player_0,killsplayer_0,riftherald_team300,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

### Try getting new patch data

In [3]:
test_list = ['NA1_4108956536']

In [5]:
#Final iteration
username = 'new_patch_data'

#Call the get_match_info function
df = get_match_info(test_list, api_key, username)

Created timeline_data_new_patch_data.json
Created other_game_data_new_patch_data.json

Preparing the data...



NameError: name 'prepare' is not defined

In [6]:
#Got an error, but that was expected. It still saved the json files
timeline_data = pd.read_json('timeline_data_new_patch_data.json')
game_data = pd.read_json('other_game_data_new_patch_data.json')

In [7]:
timeline_data = timeline_data.to_dict(orient = 'records')
game_data = game_data.to_dict(orient = 'records')

The following data is for the newest patch. __Game Version: 11.23.409.111__

In [11]:
timeline_data

[{'metadata': {'dataVersion': '2',
   'matchId': 'NA1_4108956536',
   'participants': ['eef0je_oIiZyinX0LqmevzPDSL1dO0glLHHJdzErfohEy7z6_hsg5Ve-2-pbHdi-x8agE78MbatEfg',
    'SBR-aogcZCtbAEVWMZtVIFyuvEACd04E5NqAu_5LKcQpMPk_8rfrhmxxM5EhF_heUV4SmIFmofzy0g',
    'WWIVGxkJ-4o4McCw2INDfJJOvyKn8ogqMXFERbFQyyvehgiEGplyF9mAqaiwJQow9zLIgKbbN3GQRA',
    '97SF-UR0TpY7puewHpymaQugie3-7bqgDvfJNfJ5s0SzWoyythP84wYM0Ji0BOzStHsr8rPwFDTBwA',
    'aHdOoRlQ9i1vdsD4k8Y-7bOYgS0qMycwgrJ6-Zg8KCFhhNOcXqzHdy1jSnQ2oLNfvwC6n5S6ZppEZg',
    'dxSlWFG4pwkcDhckjvmYosWZE8grOcXWEh0aq2P_0aBf2losBbTEtLCXfK39fqoOcgntF0xzG2x16A',
    '8KcTNw2bdOkcSoRdp8leGp-orNTI54JMdbb93o_UZQfsIVoyIQuP3U3SimgSaxCoFaQo0mm6FGwkJg',
    'eiW6aBbjWse8r0nhB511OB1er2vH31sfTLrwu3tK9xQdvAsV0rbzlhGJdTdqhPkKJa4HenoIv9qXgg',
    'KdCN-NhGNDTRnBnfmRgRUX6QOabrXwDH8sX4hw0-2rUf1-c0ux5iL037uAmzQkLknXkgjhX6ovdtoA',
    'IktnF9bTaXsTZd83JZCbImk8HXi5tT4NCG8UCtscQwryDxVUVOAdkvadWcIczmFwdpST9FJDI5rS_g']},
  'info': {'frameInterval': 60000,
   'frames': [{'even

In [12]:
game_data

[{'metadata': {'dataVersion': '2',
   'matchId': 'NA1_4108956536',
   'participants': ['eef0je_oIiZyinX0LqmevzPDSL1dO0glLHHJdzErfohEy7z6_hsg5Ve-2-pbHdi-x8agE78MbatEfg',
    'SBR-aogcZCtbAEVWMZtVIFyuvEACd04E5NqAu_5LKcQpMPk_8rfrhmxxM5EhF_heUV4SmIFmofzy0g',
    'WWIVGxkJ-4o4McCw2INDfJJOvyKn8ogqMXFERbFQyyvehgiEGplyF9mAqaiwJQow9zLIgKbbN3GQRA',
    '97SF-UR0TpY7puewHpymaQugie3-7bqgDvfJNfJ5s0SzWoyythP84wYM0Ji0BOzStHsr8rPwFDTBwA',
    'aHdOoRlQ9i1vdsD4k8Y-7bOYgS0qMycwgrJ6-Zg8KCFhhNOcXqzHdy1jSnQ2oLNfvwC6n5S6ZppEZg',
    'dxSlWFG4pwkcDhckjvmYosWZE8grOcXWEh0aq2P_0aBf2losBbTEtLCXfK39fqoOcgntF0xzG2x16A',
    '8KcTNw2bdOkcSoRdp8leGp-orNTI54JMdbb93o_UZQfsIVoyIQuP3U3SimgSaxCoFaQo0mm6FGwkJg',
    'eiW6aBbjWse8r0nhB511OB1er2vH31sfTLrwu3tK9xQdvAsV0rbzlhGJdTdqhPkKJa4HenoIv9qXgg',
    'KdCN-NhGNDTRnBnfmRgRUX6QOabrXwDH8sX4hw0-2rUf1-c0ux5iL037uAmzQkLknXkgjhX6ovdtoA',
    'IktnF9bTaXsTZd83JZCbImk8HXi5tT4NCG8UCtscQwryDxVUVOAdkvadWcIczmFwdpST9FJDI5rS_g']},
  'info': {'gameCreation': 1637297910000,
   'gameDurat